In [16]:
import pandas as pd
import csv

# Provided DataFrame
fastfood_df = pd.read_csv('./Resources/McDonald_s_Reviews.csv', encoding='Latin1')

In [17]:
fastfood_df['store_name'] = fastfood_df['store_name'].replace("ýýýMcDonald's", "McDonald's")


In [20]:
# Create a DataFrame from the sample data
df = pd.DataFrame(fastfood_df)

In [21]:
# First table - store_names
store_names = df[['store_name']].drop_duplicates().reset_index(drop=True)
store_names['id_store_names'] = store_names.index
# Use encode('utf-8') to handle special characters in store names
store_names['store_name'] = store_names['store_name'].str.encode('utf-8').str.decode('utf-8')
store_names = store_names[['id_store_names', 'store_name']]  # Corrected order
store_names.to_csv('store_names.csv', index=False)

In [23]:
# Second table - categories
categories = df[['category']].drop_duplicates().reset_index(drop=True)
categories['id_categories'] = categories.index
categories = categories[['id_categories', 'category']]  # Corrected order
categories.to_csv('categories.csv', index=False)

In [25]:
# Fourth table - stores
stores = df[['store_name', 'category', 'store_address', 'latitude ', 'longitude', 'rating_count']].drop_duplicates().reset_index(drop=True)
stores = stores.merge(store_names, on='store_name').merge(categories, on='category')
# Convert rating_count to float with proper format
stores['rating_count'] = stores['rating_count'].str.replace(',', '').astype(float)
# Use groupby to calculate the mean rating_count for each store_address
stores = stores.groupby(['store_address', 'id_store_names', 'id_categories', 'latitude ', 'longitude'])[['rating_count']].mean().reset_index()
stores['id_store'] = stores.index  # Create id_store as a unique identifier for each store
stores = stores[['id_store', 'id_store_names', 'id_categories', 'store_address', 'latitude ', 'longitude', 'rating_count']]  # Corrected order
stores.to_csv('stores.csv', index=False)

In [28]:
# Third table - reviews
reviews = df[['store_address', 'review_time', 'rating', 'review']].drop_duplicates().merge(stores, on='store_address', how='inner').drop_duplicates().reset_index(drop=True)

In [30]:
reviews['id_review'] = reviews.index  # Create id_review as a unique identifier for each review
reviews = reviews[['id_review', 'id_store', 'review_time', 'rating', 'review']]  # Corrected order
reviews.to_csv('reviews.csv', index=False, line_terminator='\n')
